<a href="https://colab.research.google.com/github/zhanna-naumenko/DI-Bootcamp/blob/main/W8D3_DailyChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pipq install nltk
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install wordcloud matplotlib

/bin/bash: line 1: pipq: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [4]:
!pip install tensorflow

In [5]:
import re
import requests
from tensorflow.keras.preprocessing.text import Tokenizer

# 1. Access the text
url = "https://www.gutenberg.org/cache/epub/11/pg11.txt"
response = requests.get(url)
text = response.text

# 2. Create a preprocess function to clean non-words not relevant to the context using regular expressions
def preprocess_text(text):
    # Remove non-word characters and extra whitespaces
    cleaned_text = re.sub(r'\W+', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

# 3. Split the text
def split_text(text):
    # Split text into sentences
    sentences = text.split(".")
    return sentences

# 4. Print the first 200 characters of the corpus
corpus = preprocess_text(text)
print("First 200 characters of the corpus:")
print(corpus[:200])

# 5. Using Tokenizer(), create the vocabulary and a variable called total_words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1
print("Total words in the vocabulary:", total_words)

First 200 characters of the corpus:
 The Project Gutenberg eBook of Alice s Adventures in Wonderland This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restr
Total words in the vocabulary: 3032


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert text to sequences using the tokenizer
sequences = tokenizer.texts_to_sequences([corpus])[0]

# Define the n-gram sequence length
seq_length = 10

# Create n-gram sequences
input_sequences = []
for i in range(seq_length, len(sequences)):
    seq = sequences[i - seq_length:i + 1]
    input_sequences.append(seq)

# Pad sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

print("Example of an input sequence:")
print(input_sequences[0])

print("Padded input sequences:")
print(input_sequences_padded)


Example of an input sequence:
[1, 49, 45, 307, 5, 12, 19, 358, 11, 534, 23]
Padded input sequences:
[[   1   49   45 ...   11  534   23]
 [  49   45  307 ...  534   23  307]
 [  45  307    5 ...   23  307   34]
 ...
 [   2   68    3 ...    3  267   42]
 [  68    3 3030 ...  267   42  481]
 [   3 3030    3 ...   42  481  594]]


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Convert input_sequences to a numpy array
input_sequences = np.array(input_sequences)

# Predictors and label
X = input_sequences[:-1]
labels = input_sequences[:-1]


# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Define the model
model = Sequential()

# Add an Embedding layer for text representation
embedding_dim = 100
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_length))

# Add LSTM layers for processing the sequences
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))  # Dropout layer for regularization
model.add(LSTM(units=64))
model.add(Dropout(0.2))  # Dropout layer for regularization

# Add a Dense layer for output prediction
model.add(Dense(units=total_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 100)           303200    
                                                                 
 lstm (LSTM)                 (None, 11, 64)            42240     
                                                                 
 dropout (Dropout)           (None, 11, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3032)              197080    
                                                                 
Total params: 575544 (2.20 MB)
Trainable params: 575544 

In [9]:
def generate_text(model, tokenizer, seed_text, next_words, max_sequence_length):
    output_text = seed_text
    for _ in range(next_words):
        # Tokenize the seed text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        # Predict the next word
        predicted = np.argmax(model.predict(token_list), axis=-1)
        # Convert the predicted word index to the actual word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        output_text += " " + output_word
        seed_text += " " + output_word
    return output_text

# Example usage:
seed_text = "alice was"
next_words = 10
generated_text = generate_text(model, tokenizer, seed_text, next_words, max_sequence_length + 1)
print("Generated Text:", generated_text)


1/1 [==============================] - 0s 23ms/step
Generated Text: alice was gardeners gardeners cool donors donors donors happening happening meat meat
